In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import kuaizi as kz
from astropy.table import Table
import astropy.units as u

from kuaizi.display import display_single
from astropy.io import fits
import os
import sys
from shutil import copyfile

In [4]:
kz.utils.set_env(project='HSC', name='LSBG', data_dir='/scratch/gpfs/jiaxuanl/Data/')

'/scratch/gpfs/jiaxuanl/Data/HSC/LSBG'

In [5]:
cat = Table.read('./Catalog/NSA/z002_004/nsa_cutout_cat_z002_004.fits')

### Generate RGB images

In [6]:
from kuaizi.display import display_HSC_cutout_rgb

In [7]:
CUTOUT_DIR = '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Cutout/NSA/'
FIGURE_DIR = '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/'
WEBPAGE_FIG_DIR = '/tigress/jiaxuanl/public_html/NSA/cutout_RGB/z002_004/figure/'

In [8]:
problem_ids = []
# Check file exsistence
for filt in 'grizy':
    for obj in cat:
        if not os.path.isfile(CUTOUT_DIR + f"nsa_{obj['viz-id']}_{filt}.fits"):
            #print(f"NSA/nsa_{obj['viz-id']}_{filt}.fits not found!")
            problem_ids.append(obj['viz-id'])
            
if not os.path.isdir(FIGURE_DIR):
    os.mkdir(FIGURE_DIR)

In [9]:
print('# of no cutouts:', np.unique(problem_ids).shape)

# of no cutouts: (72,)


In [10]:
# Display each cutout
for i, obj in enumerate(cat[1100:1200]):
    ind = obj['viz-id']
    try:
        images = np.asarray([fits.open(f'{CUTOUT_DIR}/nsa_{ind}_{filt}.fits')[1].data for filt in 'griz'])
        fig, ax = plt.subplots(figsize=(5, 5))
        #size = obj['rhalf_ellip'] * 5 if obj['rhalf_ellip'] * 5 > 100 else 100
        ax = display_HSC_cutout_rgb(images, ax=ax, half_width=None)
        plt.savefig(FIGURE_DIR + f'nsa_{ind}_cutout.png', dpi=70, bbox_inches='tight')
        plt.close()
    except Exception as e:
        print(ind, e)
        
    if i % 10 == 0:
        print(f'Progress: {i} / {len(cat)}')

Progress: 0 / 5576
Progress: 10 / 5576
Progress: 20 / 5576
Progress: 30 / 5576
Progress: 40 / 5576
Progress: 50 / 5576
18082 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Cutout/NSA//nsa_18082_g.fits'
Progress: 60 / 5576
18268 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Cutout/NSA//nsa_18268_i.fits'
Progress: 70 / 5576
Progress: 80 / 5576
Progress: 90 / 5576


In [11]:
# Copy figures in FIGURE_DIR to WEBPAGE_FIG_DIR
if not os.path.isdir(WEBPAGE_FIG_DIR):
    os.mkdir(WEBPAGE_FIG_DIR)
else:
    import glob
    fileList = glob.glob(WEBPAGE_FIG_DIR + '*.png', recursive=True)
    [os.remove(file) for file in fileList];
            
for i, obj in enumerate(cat[1100:1200]):
    ind = obj['viz-id']
    try: 
        copyfile(FIGURE_DIR + f'nsa_{ind}_cutout.png', WEBPAGE_FIG_DIR + f'nsa_{ind}_cutout.png')
    except Exception as e:
        print(ind, e)

18082 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_18082_cutout.png'
18268 [Errno 2] No such file or directory: '/scratch/gpfs/jiaxuanl/Data/HSC/LSBG/Figure/NSA/Cutout_RGB/nsa_18268_cutout.png'


### Generate webpage for cutouts. We use `diezi/webpage/gen_web_cutout.py`

In [12]:
sys.path.append('/home/jiaxuanl/Research/Packages/kuaizi/diezi/webpage/')
from gen_web_cutout import webpage_cutout

In [13]:
#FOR NSA SAMPLE
WEBPAGE_FIG_DIR = '/tigress/jiaxuanl/public_html/NSA/cutout_RGB/z002_004/figure/'
WEBPAGE_DIR = '/tigress/jiaxuanl/public_html/NSA/cutout_RGB/z002_004/page/'
SCARLET_DIR = None #'/tigress/jiaxuanl/public_html/NSA/scarlet_result/'
sample_name = 'NSA'

In [14]:
webpage_cutout(WEBPAGE_FIG_DIR, WEBPAGE_DIR, sample_name, SCARLET_DIR=SCARLET_DIR, 
               col_num=6, row_num=4)

You have 98 galaxies to be displayed
Total pages: 5


In [15]:
print('The webpages are in', WEBPAGE_DIR)

The webpages are in /tigress/jiaxuanl/public_html/NSA/cutout_RGB/z002_004/page/


### The search box on webpage still doesn't work. Need to work on the javascript file